## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


# Roshni's Scratch Notebook

# 1. "Rotten Tomatoes: Movie Info" Dataset

In [83]:
import pandas as pd

rt_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz', sep = '\t')

rt_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


# Examine Columns/Variables of Interest

In [84]:
rt_movie_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


### Notes:
- Movie title is not included in this database, just ID.
 - So much missing data!!! (more on that below)
 
### Potential Predictor Variables
 1. MPAA Rating
 2. Genre
 3. Director
 4. Writer
 5. Theater Date (month released/time of month?)
 6. Runtime

### Potential Outcome Variable
 1. Box_Office
 - Seems to be 1st weekend collection in US only? 
 - Probably not the best source to use
 
 2. Currency (only USD, useless)
 
### Connector Variables
 1. id


## Missing Data: 10% Rule

In [85]:
# Columns that have more than 10% of data missing:

print(1560*10/100) 

#156 Missing, 1404 Present

((rt_df.isna().sum())>156)

#Director, Writer, Theater_Date, DVD_Date, Currency, Box_Office, Studio

156.0


id              False
synopsis        False
rating          False
genre           False
director         True
writer           True
theater_date     True
dvd_date         True
currency         True
box_office       True
runtime         False
studio           True
dtype: bool

## Descriptive Statistics

- Numeric variables: Box_Office, Runtime
- Categorical variables: the rest
- Theater_Date --> convert to DateTime if interesting

In [86]:
rt_df['rating'].value_counts(normalize = True)* 100

R        33.461785
NR       32.305716
PG       15.414258
PG-13    15.093128
G         3.660886
NC17      0.064226
Name: rating, dtype: float64

In [87]:
rt_df['genre'].value_counts(normalize = True) * 100

#299 different Genres! will have to parse through & convert to lists to use?

Drama                                                                                          9.729381
Comedy                                                                                         7.087629
Comedy|Drama                                                                                   5.154639
Drama|Mystery and Suspense                                                                     4.317010
Art House and International|Drama                                                              3.994845
                                                                                                 ...   
Animation|Comedy                                                                               0.064433
Action and Adventure|Comedy|Kids and Family|Science Fiction and Fantasy|Romance                0.064433
Action and Adventure|Comedy|Kids and Family|Mystery and Suspense                               0.064433
Documentary|Special Interest|Television                         

In [88]:
rt_df['runtime'].value_counts()

# need to get rid of minutes, need to convert to int

90 minutes     72
95 minutes     66
100 minutes    51
93 minutes     47
96 minutes     43
               ..
174 minutes     1
61 minutes      1
240 minutes     1
13 minutes      1
30 minutes      1
Name: runtime, Length: 142, dtype: int64

In [89]:
#Unique movies in dataset:

len(rt_df['id'].value_counts())

1560

<br>

# 2. "Rotten Tomatoes: Reviews" Dataset 

In [90]:
import pandas as pd

rt_rev_df = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep = '\t', encoding = 'unicode_escape', error_bad_lines=False)


rt_rev_df.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


### Notes:
 - Still no movie title! What does ID link up to then?
 - 1 Row represents 1 Critic Movie Review
**Multiple Rows link up to 1 Movie** so for analysis, need to use some form of aggregatation

### Potential Predictor Variables:
 - Aggregate Rating (Avg/Median/etc.)
 - Aggregate Fresh vs. Rotten 
 - Top_Critic status x Rating (e.g. anything top critics rate as Fresh are box office hits)
 - Specific Critic x Rating (e.g. anything PJ Nabarro likes is a box office hit)
 
 ### Connector Variables:
  - id

In [91]:
rt_rev_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


### Notes:

- Significantly more data than 1st dataset (as is to be expected, multiple reviews per movie)

### 10% rule:

Missing data > 5443

In [92]:
((rt_rev_df.isna().sum())>5443)

# Rating

id            False
review         True
rating         True
fresh         False
critic        False
top_critic    False
publisher     False
date          False
dtype: bool

### How many unique movies are represented in this dataset?

In [93]:
len(rt_rev_df['id'].value_counts()) # 1135

1135

- 1135 movies in this dataset vs. 1560 movies in "Movie Info" dataset?

- Missing review data on **325 movies**?

## Descriptive Statistics

**Categorical Variables**: Fresh, Critic, Top_Critic

**Numeric Variables**: Rating (needs to be converted to int & standardized)


### Rating:

In [94]:
rt_rev_df['rating'].value_counts()

3/5       4327
4/5       3672
3/4       3577
2/5       3160
2/4       2712
          ... 
6.3/10       1
7.4          1
4.2          1
9.2          1
8.4          1
Name: rating, Length: 186, dtype: int64

In [95]:
rt_rev_df['fresh'].value_counts()

fresh     33035
rotten    21397
Name: fresh, dtype: int64

In [96]:
rt_rev_df['top_critic'].value_counts()

0    41336
1    13096
Name: top_critic, dtype: int64

<br>

# The Numbers Dataset

In [97]:
tn_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

tn_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [98]:
tn_df.tail()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0
5781,82,"Aug 5, 2005",My Date With Drew,"$1,100","$181,041","$181,041"


## Notes:
1. How does a movie gross nothing? 
2. Domestic Gross = total box office collections in US & Canada

### Potential Variables of Interest:
1. Release Date
2. Production Budget

### Outcome Variables:
1. Domestic Gross
2. Worldwide Gross
3. Production Budget (if we want to look at net profits)

### Connector Variables:
1. Movie Name
2. ID?

In [99]:
tn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


- No missing data! 

- All objects: need to remove commas, convert to int

In [100]:
tn_df['id'].value_counts()

4      58
53     58
61     58
65     58
69     58
       ..
85     57
100    57
96     57
92     57
97     57
Name: id, Length: 100, dtype: int64

 - ID not a unique identifier, seems to just be the last 2 digits of the original index